# ParlIA

## Instalación del ParlIA

In [17]:
!pip3 install -q parlai
!pip3 install -q subword_nmt 

## Importando el Dataset

### Importamos el Dataset.

In [18]:
from parlai.core.build_data import DownloadableFile
import parlai.core.build_data as build_data
import os

RESOURCES = [
    DownloadableFile(
        'http://parl.ai/downloads/cbt/cbt.tar.gz',
        'cbt.tar.gz',
        '932df0cadc1337b2a12b4c696b1041c1d1c6d4b6bd319874c6288f02e4a61e92',
    )
]


def build(opt):
    dpath = os.path.join(opt['datapath'], 'CBT')
    version = None

    if not build_data.built(dpath, version_string=version):
        print('[building data: ' + dpath + ']')
        if build_data.built(dpath):
            # Se elimina los archivos desactualizados de la versión anterior.
            build_data.remove_dir(dpath)
        build_data.make_dir(dpath)

        # Descarga los datos.
        for downloadable_file in RESOURCES:
            downloadable_file.download_file(dpath)

        # Marca los datos como construidos.
        build_data.mark_done(dpath, version_string=version)

### Mostramos un ejemplo

In [28]:
# El script display_data se usa para mostrar el contenido de una tarea en particular.
# Por defecto, mostramos el train
from parlai.scripts.display_data import DisplayData
DisplayData.main(task='cbt', num_examples=1)

06:20:10 | Opt:
06:20:10 |     allow_missing_init_opts: False
06:20:10 |     batchsize: 1
06:20:10 |     datapath: C:\Users\hp\anaconda3\envs\pytorch\lib\site-packages\data
06:20:10 |     datatype: train:ordered
06:20:10 |     dict_class: None
06:20:10 |     display_add_fields: 
06:20:10 |     download_path: None
06:20:10 |     dynamic_batching: None
06:20:10 |     hide_labels: False
06:20:10 |     ignore_agent_reply: True
06:20:10 |     image_cropsize: 224
06:20:10 |     image_mode: raw
06:20:10 |     image_size: 256
06:20:10 |     init_model: None
06:20:10 |     init_opt: None
06:20:10 |     is_debug: False
06:20:10 |     loglevel: info
06:20:10 |     max_display_len: 1000
06:20:10 |     model: None
06:20:10 |     model_file: None
06:20:10 |     multitask_weights: [1]
06:20:10 |     mutators: None
06:20:10 |     num_examples: 1
06:20:10 |     override: "{'task': 'cbt', 'num_examples': 1}"
06:20:10 |     parlai_home: C:\Users\hp\anaconda3\envs\pytorch\lib\site-packages
06:20:10 |     

### Implementamos los agentes.

In [19]:
from parlai.core.teachers import FbDeprecatedDialogTeacher, MultiTaskTeacher

import copy
import os


def _path(task, opt):
    # Genera los datos si no existen.
    build(opt)
    suffix = ''
    dt = opt['datatype'].split(':')[0]
    if dt == 'train':
        suffix = 'train'
    elif dt == 'test':
        suffix = 'test_2500ex'
    elif dt == 'valid':
        suffix = 'valid_2000ex'

    return os.path.join(
        opt['datapath'], 'CBT', 'CBTest', 'data', task + '_' + suffix + '.txt'
    )


class NETeacher(FbDeprecatedDialogTeacher):
    def __init__(self, opt, shared=None):
        opt['datafile'] = _path('cbtest_NE', opt)
        opt['cloze'] = True
        super().__init__(opt, shared)


class CNTeacher(FbDeprecatedDialogTeacher):
    def __init__(self, opt, shared=None):
        opt['datafile'] = _path('cbtest_CN', opt)
        opt['cloze'] = True
        super().__init__(opt, shared)


class VTeacher(FbDeprecatedDialogTeacher):
    def __init__(self, opt, shared=None):
        opt['datafile'] = _path('cbtest_V', opt)
        opt['cloze'] = True
        super().__init__(opt, shared)


class PTeacher(FbDeprecatedDialogTeacher):
    def __init__(self, opt, shared=None):
        opt['datafile'] = _path('cbtest_P', opt)
        opt['cloze'] = True
        super().__init__(opt, shared)


# De forma predeterminada, entrena en todas las tareas a la vez.

class DefaultTeacher(MultiTaskTeacher):
    def __init__(self, opt, shared=None):
        opt = copy.deepcopy(opt)
        opt['task'] = 'cbt:NE,cbt:CN,cbt:V,cbt:P'
        super().__init__(opt, shared)

### Anadir el task al ParlAI

In [21]:
from parlai.utils.testing import AutoTeacherTest  

class TestDefaultTeacher(AutoTeacherTest):
    task = 'cbt'

# Creando nuestro modelo

In [22]:
import torch.nn as nn
import torch.nn.functional as F
import parlai.core.torch_generator_agent as tga


class Encoder(nn.Module):
    """
    Codificador de ejemplo, que consta de una capa de incrustación y un LSTM de 1 capa con el
    tamaño oculto especificado.
    """

    def __init__(self, embeddings, hidden_size):
        """
        Inicialización.
        Los argumentos aquí se pueden utilizar para proporcionar hiperparámetros.
        """
        # Llama a super en todos los nn.Modules para que lo herede.
        super().__init__()

        self.embeddings = embeddings
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
        )

    def forward(self, input_tokens):
        """            
        Realice el forward pass para el codificador.
        La entrada debe ser input_tokens, que son los tokens de contexto dados
        como una matriz de ID de búsqueda.
        : param input_tokens:
            Introduzca tokens como bsz x seqlen LongTensor.
            Probablemente contendrá relleno.
        :regreso:
            Puede devolver lo que quiera; se pasará palabra por palabra
            en el decodificador para acondicionamiento. Sin embargo, debería ser algo
            puede manipular fácilmente en '' reorder_encoder_states ''.
            Esta implementación particular devuelve los estados ocultos y de celda de la
            LSTM.
        """
        embedded = self.embeddings(input_tokens)
        _output, hidden = self.lstm(embedded)
        return hidden


class Decoder(nn.Module):
    """
    Decodificador de ejemplo básico, que consta de una capa de incrustación y un LSTM de 1 capa con el
    tamaño oculto especificado. El decodificador permite la decodificación incremental ingiriendo el
    estado incremental actual en cada pasada hacia adelante.
    Preste especial atención al "forward".
    
    """

    def __init__(self, embeddings, hidden_size):
        """
        Inicialización.
        Los argumentos aquí se pueden utilizar para proporcionar hiperparámetros.
        """
        super().__init__()
        self.embeddings = embeddings
        self.lstm = nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
        )

    def forward(self, input, encoder_state, incr_state=None):
        """
        Correr el forward pass.
        :parámetro input:
            Los tokens generados actualmente por el decodificador.
        :parámetro encoder_state:
            La salida del módulo codificador.
        :parámetro incr_state:
            El estado oculto anterior del decodificador. 
        """
        embedded = self.embeddings(input)
        if incr_state is None:
            # esta es nuestra primera llamada. Queremos sembrar el LSTM con el estado oculto del decodificador.
            state = encoder_state
        else:
            # Ya hemos generado algunos tokens, por lo que podemos reutilizar el estado del decodificador existente
            state = incr_state

        # obtener la nueva salida y el estado incremental del decodificador
        output, incr_state = self.lstm(embedded, state)

        return output, incr_state


class ExampleModel(tga.TorchGeneratorModel):
    """
    ExampleModel implementa los métodos abstractos de TorchGeneratorModel para definir cómo
    reordenar los estados del codificador y los estados incrementales del descodificador.
    También crea una instancia de la tabla de incrustación, el codificador y el descodificador, y define el
    capa de salida final.
    """

    def __init__(self, dictionary, hidden_size=1024):
        super().__init__(
            padding_idx=dictionary[dictionary.null_token],
            start_idx=dictionary[dictionary.start_token],
            end_idx=dictionary[dictionary.end_token],
            unknown_idx=dictionary[dictionary.unk_token],
        )
        self.embeddings = nn.Embedding(len(dictionary), hidden_size)
        self.encoder = Encoder(self.embeddings, hidden_size)
        self.decoder = Decoder(self.embeddings, hidden_size)

    def output(self, decoder_output):
        """
        Realiza la salida final -> transformación logits.
        """
        return F.linear(decoder_output, self.embeddings.weight)

    def reorder_encoder_states(self, encoder_states, indices):
        """
        Reordena los estados del codificador para seleccionar solo los índices de lote dados.
        Se indexa la selección en la dimensión del lote.
        """
        h, c = encoder_states
        return h[:, indices, :], c[:, indices, :]

    def reorder_decoder_incremental_state(self, incr_state, indices):
        """
        Este método puede ser un código auxiliar que siempre devuelve None; esto resultará en el
        decodificador haciendo un pase completo hacia adelante para cada token.
        Sin embargo, si cualquier estado se puede almacenar en caché, este método debe ser
        implementado para reducir la complejidad de generación.
        """
        h, c = incr_state
        return h[:, indices, :], c[:, indices, :]


@register_agent("my_first_lstm")
class Seq2seqAgent(tga.TorchGeneratorAgent):
    """
    Implementa la interfaz para TorchGeneratorAgent.
    Implementa `` build_model '', se incluye una línea de comando adicional
    parámetros.
    """

    @classmethod
    def add_cmdline_args(cls, argparser, partial_opt):
        """       
        Agregar argumentos CLI.
        """
        # Agrega todos los argumentos de TorchGeneratorAgent
        super().add_cmdline_args(argparser)

        # Agregamos argumentos personalizados solo para este modelo.
        group = argparser.add_argument_group('Example TGA Agent')
        group.add_argument(
            '-hid', '--hidden-size', type=int, default=1024, help='Hidden size.'
        )

    def build_model(self):
        """  
        Construye el modelo.
        """

        model = ExampleModel(self.dict, self.opt['hidden_size'])
        self._copy_embeddings(model.embeddings.weight, self.opt['embedding_type'])
        return model

## Entrenando el Modelo


In [23]:
from parlai.scripts.train_model import TrainModel
from parlai.core.agents import create_agent

TrainModel.main(
    model='my_first_lstm',
    model_file='my_first_lstm/model',
    task='cbt',
    batchsize=1,
    validation_every_n_secs=10,
    max_train_time=60,
)

03:49:52 | building dictionary first...
03:49:52 | No model with opt yet at: my_first_lstm/model(.opt)
03:49:52 | Using CUDA
03:49:52 | loading dictionary from my_first_lstm/model.dict
03:49:52 | num words = 51210
03:49:52 | Total parameters: 69,232,640 (69,232,640 trainable)
03:49:52 | Opt:
03:49:52 |     adafactor_eps: '(1e-30, 0.001)'
03:49:52 |     adam_eps: 1e-08
03:49:52 |     add_p1_after_newln: False
03:49:52 |     aggregate_micro: False
03:49:52 |     allow_missing_init_opts: False
03:49:52 |     batchsize: 1
03:49:52 |     beam_block_full_context: True
03:49:52 |     beam_block_list_filename: None
03:49:52 |     beam_block_ngram: -1
03:49:52 |     beam_context_block_ngram: -1
03:49:52 |     beam_delay: 30
03:49:52 |     beam_length_penalty: 0.65
03:49:52 |     beam_min_length: 1
03:49:52 |     beam_size: 1
03:49:52 |     betas: '(0.9, 0.999)'
03:49:52 |     bpe_add_prefix_space: None
03:49:52 |     bpe_debug: False
03:49:52 |     bpe_dropout: None
03:49:52 |     bpe_merge: No

({'cbt:NE/exs': SumMetric(2000),
  'exs': SumMetric(8000),
  'cbt:NE/accuracy': ExactMatchMetric(0),
  'cbt:NE/f1': F1Metric(0),
  'cbt:NE/bleu-4': BleuMetric(0),
  'cbt:NE/clen': AverageMetric(489),
  'cbt:NE/ctrunc': AverageMetric(0),
  'cbt:NE/ctrunclen': AverageMetric(0),
  'cbt:NE/llen': AverageMetric(2.047),
  'cbt:NE/ltrunc': AverageMetric(0),
  'cbt:NE/ltrunclen': AverageMetric(0),
  'cbt:NE/loss': AverageMetric(10.06),
  'cbt:NE/ppl': PPLMetric(2.333e+04),
  'cbt:NE/token_acc': AverageMetric(0.2694),
  'cbt:NE/token_em': AverageMetric(0),
  'cbt:CN/exs': SumMetric(2000),
  'cbt:CN/accuracy': ExactMatchMetric(0.001),
  'cbt:CN/f1': F1Metric(0.001),
  'cbt:CN/bleu-4': BleuMetric(1e-12),
  'cbt:CN/clen': AverageMetric(522.1),
  'cbt:CN/ctrunc': AverageMetric(0),
  'cbt:CN/ctrunclen': AverageMetric(0),
  'cbt:CN/llen': AverageMetric(2.004),
  'cbt:CN/ltrunc': AverageMetric(0),
  'cbt:CN/ltrunclen': AverageMetric(0),
  'cbt:CN/loss': AverageMetric(11.07),
  'cbt:CN/ppl': PPLMetric(

In [24]:
TrainModel.main(
    model='my_first_lstm',
    model_file='my_first_lstm/model',
    task='cbt',
    batchsize=3,
    validation_every_n_secs=10,
    max_train_time=60,
)

05:15:31 | building dictionary first...
05:15:31 | Overriding opt["batchsize"] to 3 (previously: 1)
05:15:31 | Using CUDA
05:15:31 | loading dictionary from my_first_lstm/model.dict
05:15:31 | num words = 51210
05:15:32 | Total parameters: 69,232,640 (69,232,640 trainable)
05:15:32 | Loading existing model params from my_first_lstm/model
05:15:32 | Opt:
05:15:32 |     adafactor_eps: '[1e-30, 0.001]'
05:15:32 |     adam_eps: 1e-08
05:15:32 |     add_p1_after_newln: False
05:15:32 |     aggregate_micro: False
05:15:32 |     allow_missing_init_opts: False
05:15:32 |     batchsize: 3
05:15:32 |     beam_block_full_context: True
05:15:32 |     beam_block_list_filename: None
05:15:32 |     beam_block_ngram: -1
05:15:32 |     beam_context_block_ngram: -1
05:15:32 |     beam_delay: 30
05:15:32 |     beam_length_penalty: 0.65
05:15:32 |     beam_min_length: 1
05:15:32 |     beam_size: 1
05:15:32 |     betas: '[0.9, 0.999]'
05:15:32 |     bpe_add_prefix_space: None
05:15:32 |     bpe_debug: Fals

({'cbt:NE/exs': SumMetric(2000),
  'exs': SumMetric(8000),
  'cbt:NE/accuracy': ExactMatchMetric(0),
  'cbt:NE/f1': F1Metric(0.00025),
  'cbt:NE/bleu-4': BleuMetric(6.767e-14),
  'cbt:NE/clen': AverageMetric(489),
  'cbt:NE/ctrunc': AverageMetric(0),
  'cbt:NE/ctrunclen': AverageMetric(0),
  'cbt:NE/llen': AverageMetric(2.047),
  'cbt:NE/ltrunc': AverageMetric(0),
  'cbt:NE/ltrunclen': AverageMetric(0),
  'cbt:NE/loss': AverageMetric(10.13),
  'cbt:NE/ppl': PPLMetric(2.509e+04),
  'cbt:NE/token_acc': AverageMetric(0.3464),
  'cbt:NE/token_em': AverageMetric(0),
  'cbt:CN/exs': SumMetric(2000),
  'cbt:CN/accuracy': ExactMatchMetric(0.001),
  'cbt:CN/f1': F1Metric(0.001),
  'cbt:CN/bleu-4': BleuMetric(1e-12),
  'cbt:CN/clen': AverageMetric(522.1),
  'cbt:CN/ctrunc': AverageMetric(0),
  'cbt:CN/ctrunclen': AverageMetric(0),
  'cbt:CN/llen': AverageMetric(2.004),
  'cbt:CN/ltrunc': AverageMetric(0),
  'cbt:CN/ltrunclen': AverageMetric(0),
  'cbt:CN/loss': AverageMetric(11.36),
  'cbt:CN/pp